In [3]:
from utils.utils import prepare_chat_history, prepare_kshot_prompt_using_levenshtein_distance, default_arguments_for_openai_validation, create_openai_completion
from utils.models_metadata import get_model_metadata

In [4]:
def get_completion_using_vtt(chat_history, question="NA", provided_answer="NA", model_name="validate_answer_to_custom_question", model_version=None, ):
    """
    Hits OpenAI's completion endpoint to validate the answer an applicant gives to a question. Returns the validation according to the completion template.

        Parameters:
                chat_history (list of strings): Conversation so far between the interviewer and applicant (first to most recent). Each string should start with either "Interviewer: " or "Applicant: ".
                question (string): Optional - The question whose answer we want to validate. If this is not passed, we validate the answer to the last question the interviewer asks in `chat_history`.
                provided_answer (string): Optional - The expected answer for `question`. If this is not passed, we hope our model is able to figure out the right answer and validate the applicant's answer accordingly.
                model_name (string): Optional - Which model from `models_metadata.json` we want to use to get a completion.
                model_version (string): Optional - The version of `model_name` we want to use. If this is empty, `get_model_metadata` figures out the best model version from `models_metadata.json` and uses it.
        Returns:
                completion (string): A completion from OpenAI that validates the applicant's answer according to the completion template in `models_metadata.json` for the given `model_name` and `model_version`.
    """
    prompt_args = dict(
                    current_section_chat = prepare_chat_history(chat_history, max_length=800, empty_value='NA'),
                    question = question,
                    provided_answer = provided_answer,
                    is_completion_correct = 1)
    model_metadata, model_version = get_model_metadata(model_name, model_version)
    observation_prompt = model_metadata['prompt_template'].format(**prompt_args)

    kshot_prompt = prepare_kshot_prompt_using_levenshtein_distance(model_name=model_name, 
                                                                model_metadata=model_metadata, 
                                                                prompt_args=prompt_args, 
                                                                observation_prompt=observation_prompt)
    completion = create_openai_completion(kshot_prompt, model_metadata=model_metadata, args=default_arguments_for_openai_validation)
    return completion


In [5]:
completion = get_completion_using_vtt(
    ["Interviewer: What are the pros and cons of a knn classifier versus a logistic regression model?",
     "Applicant: KNN is faster than logistic regression when making inferences."]
     )
print(completion)

Validation of response: 1
Is question subjective: 1

Subpoints made / expected: 
Subpoint: <KNN is faster than logistic regression when making inferences.><c><NA>
Subpoint: <KNN is more prone to overfitting than logistic regression.><m><NA>

Number of correct, missing, incorrect, irrelevant subpoints: 1, 1, 0, 0
